<a href="https://colab.research.google.com/github/Shirouky/lab6-Data-analysis/blob/main/LW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Базовые функции

In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
def import_xlsx():
  FILE_NAME = "DataFrame beads.xlsx"
  df = pd.read_excel(FILE_NAME)
  return df

df = import_xlsx()

In [ ]:
def timer(func):
  def inner(*args):
    start = time.time()
    result = func(*args)
    end = time.time()
    print(f"Время выполнения: {end - start:.4f} сек")

    return result
  return inner

# Медленные и быстрые функции

In [ ]:
@timer
def analyze_profit_slow(df, column="code_category"):
    profit = {}
    top_categories = df[column].value_counts().head(10).index
    profit = pd.DataFrame(index=top_categories)

    for category in top_categories:
        mask = df[column] == category
        category_data = df.loc[mask]

        profit.loc[category, "revenue"] = (category_data["price"] * category_data["quantity"]).sum()
        profit.loc[category, "avg_price"] = category_data["price"].mean()
        profit.loc[category, "quantity"] = category_data["quantity"].sum()
        profit.loc[category, "product_count"] = len(category_data)

    return profit

In [ ]:
@timer
def analyze_profit_fast(df, column="code_category"):
    df["revenue"] = df["price"] * df["quantity"]

    top_categories = df[column].value_counts().head(10).index
    profit = df[df[column].isin(top_categories)].groupby(column).agg({
        "revenue": "sum",
        "price": "mean",
        "quantity": "sum",
        "id_web": "count"
    }).rename(columns={"id_web": "product_count", "price": "avg_price"}).round(2)

    return profit

In [ ]:
@timer
def analyze_discount_slow(df):
  discount_effectiveness = 0
  for i in range(len(df)):
      if df.iloc[i]["sale"] < -5:
          discount_effectiveness += df.iloc[i]["price"] * df.iloc[i]["quantity"]

  return discount_effectiveness

In [ ]:
@timer
def analyze_discount_fast(df):
  discount_effectiveness = df[df["sale"] < -5]["revenue"].sum()

  return discount_effectiveness

# Вывод

In [ ]:
def print_profit(profit, title="ПРИБЫЛЬНОСТЬ ПО ТОП-10 КАТЕГОРИЯМ"):
    print(f"\n{title}")
    print('=' * 70)
    print(f"{"Категория":<12} {"Выручка, руб":<15} {"Ср. цена":<10} {"Кол-во":<8} {"Товаров":<8}")
    print('-' * 70)

    for category, metrics in profit.iterrows():
        revenue = f"{metrics["revenue"]:,.0f}"
        avg_price = f"{metrics["avg_price"]:.0f}"
        quantity = f"{metrics["quantity"]:,.0f}"
        product_count = f"{metrics["product_count"]:,.0f}"
        print(f"{category:<12} {revenue:<15} {avg_price:<10} {quantity:<8} {product_count:<8}")

In [ ]:
def print_summary(profit, discount):
    total_revenue = profit["revenue"].sum()
    discount_percentage = (discount / total_revenue) * 100

    best_revenue_category = profit.loc[profit["revenue"].idxmax()]

    print(f"Общая выручка топ-10 категорий: {total_revenue} руб")
    print(f"Доля продаж со скидкой: {discount_percentage:.1f}%")
    print(f"Категория с максимальной выручкой: {best_revenue_category.name} ({best_revenue_category['revenue']:,.0f} руб)")

In [ ]:
def compare():
    df = import_xlsx()

    print("МЕДЛЕННАЯ ВЕРСИЯ:")
    profit_slow = analyze_profit_slow(df)
    discount_slow = analyze_discount_slow(df)
    print_profit(profit_slow)
    print(f"\nВыручка от товаров со скидкой >5%: {discount_slow} руб")

    print("БЫСТРАЯ ВЕРСИЯ:")
    profit_fast = analyze_profit_fast(df)
    discount_fast = analyze_discount_fast(df)
    print_profit(profit_fast)
    print(f"\nВыручка от товаров со скидкой >5%: {discount_fast} руб")
    return profit_fast, discount_fast

In [ ]:
profit, discount = compare()
print_summary(profit, discount)

МЕДЛЕННАЯ ВЕРСИЯ:
Время выполнения: 0.0623 сек
Время выполнения: 0.2855 сек

ПРИБЫЛЬНОСТЬ ПО ТОП-10 КАТЕГОРИЯМ
Категория    Выручка, руб    Ср. цена   Кол-во   Товаров 
----------------------------------------------------------------------
24           409,610         167        2,392    149     
25           195,284         207        979      72      
5            208,763         211        985      65      
2            139,242         178        785      62      
20           158,962         176        918      54      
26           125,422         172        766      41      
63           164,665         194        829      39      
18           70,962          159        457      35      
59           78,644          179        439      33      
29           69,946          147        468      30      

Выручка от товаров со скидкой >5%: 3105793 руб
БЫСТРАЯ ВЕРСИЯ:
Время выполнения: 0.0209 сек
Время выполнения: 0.0014 сек

ПРИБЫЛЬНОСТЬ ПО ТОП-10 КАТЕГОРИЯМ
Категория    Выручка, р